# Problem Set #5

In problem set #5, you are asked to use Cantera to perform kinetic simulations and analysis.
This notebook will be used as a template to pose questions and guide you through the process to reach solutions.

This assignment directly apply material presented in the two Jupyter tutorials presenting in problem sessions 2 and 5.
It is *highly* recommended that, if you haven't already, you watch the recordings and/or work through these tutorials
 before attempting this assignment.

This assignment is intended to expose students to open-source tools and analysis methods for performing kinetic
 simulations, not to test student's programming ability.
While we expect students to first reference the tutorials and documentation available to you, please do not hesitate to
 reach out to your fellow students (as with all problem sets, you are encouraged to work together but must complete and
 submit your own assignments) or your TA's.

*Note:*
Please make sure to refer to the directions at the end of Tutorial #1 regarding saving your work in the
 Jupyter/Binder environment. Please be sure to save your work to your local machine in the **.ipynb** format regularly.
If contacting the TA's by email with questions, please attach your .ipynb file to help them assist you with
 troubleshooting

**Submission:**
When you complete the assignment, please *Print* the notebook (with all output displayed) in **PDF** format for
 submission on Gradescope.
If you choose to complete the assignment in a different form than this notebook (allowed, but *not recommended*), you
 are responsible for determining what deliverables are being asked for in this template and making sure they are
 submitted in an appropriate format.

**Import Statements:**
Do not change anything in the following cell. Packages imported here should be the only required for completing this
 assignment.

In [2]:
import cantera as ct
import numpy as np
import os
import matplotlib.pyplot as plt

**Useful Functions from Problem Session #5/Tutorial #2:**

The following functions that were developed in Tutorial #2 are provided for you here for your reference and use in this
 problem set.
You may modify them should you so choose, or use them as they are.

In [ ]:
# default parameters passed to functions:
#   You can change these, then re-run this cell to change the defaults used by the functions
T0 = 1200  # K
P0 = 101325  # Pa (1 atm)
phi = 1.
fuel = {'H2':1.}
oxidizer = {'AR':.79, 'O2':.21}  # "airgon"

# user-defined functions to help set up and run simulations

def setup_simulation(sln, T=T0, P=P0, phi=phi, f=fuel, ox=oxidizer):
    sln.set_equivalence_ratio(phi=phi, fuel=f, oxidizer=ox)
    sln.TP = T, P
    r1 = ct.ConstPressureReactor(sln)
    rnet = ct.ReactorNet([r1])
    return r1, rnet


# Part 1) Comparing Mechanisms

There remains a surprising degree of uncertainty in even the simplest of kinetic mechanisms. In this part, you are asked
 to conduct simulations with three kinetic simulations and compare their results.

The following cell initializes a dictionary of the paths to the three mechanisms you will use.
It also demonstrates how to initialize `ct.Solution` objects using those paths and displays some details about the
 mechanisms.

No change is needed in this cell, but the code it contains may help you think about how to implement future steps.

In [45]:
'''No change needed in this cell'''

mech_files = {'Stanford':os.path.abspath('../mechanisms/stanford-mech_v12.cti'),
              'GRI 3.0':'gri30.cti',
              'FFCM-1':os.path.abspath('../mechanisms/FFCM1.cti'),
              'USC-Mech II':os.path.abspath('../mechanisms/USC_MechII.cti')}
mech_list = list(mech_files.keys())

for mech in mech_list:
    gas = ct.Solution(mech_files[mech])
    print(f'{mech + ":":12s}\t{gas.n_species:3.0f} Species\t{gas.n_reactions} Reactions')

Stanford:   	 10 Species	31 Reactions
GRI 3.0:    	 53 Species	325 Reactions
FFCM-1:     	 38 Species	291 Reactions
USC-Mech II:	111 Species	784 Reactions


In [46]:
'''No change needed in this cell, which defines a function to initialize a dictionary in which to store results'''

def init_result_dict(gas: ct.Solution, species_of_interest='all') -> dict:
    """
    Initializes and returns a dictionary formatted to store simulation results
    :param gas: ct.Solution to be used in the simulation
    :param species_of_interest: list of species to track, default 'all' to track all species in mechanism
    :return result_dict: dictionary with empty lists for the time, temperature, and species of interest
    """
    results = {'mechanism name':gas.name,
              'time':[],
               'T':[]}
    if species_of_interest == 'all':
        species_list = gas.species_names
    else:
        mech_species = gas.species_names
        species_list = [sp for sp in species_of_interest if sp in mech_species]
    for sp in species_list:
        results[sp] = []
    results['species_list'] = species_list
    results['species_indices'] = np.array([gas.species_index(sp) for sp in species_list])
    return results

In the following cell, we can see the format of the dictionary we'll be using to store simulation results

In [43]:
'''No change needed in this cell'''

gas = ct.Solution(mech_files['GRI 3.0'])
init_result_dict(gas, species_of_interest=['H2', 'O2', 'OH', 'O', 'H', 'HO2', 'H2O2'])

{'mechanism name': 'gri30',
 'time': [],
 'T': [],
 'H2': [],
 'O2': [],
 'OH': [],
 'O': [],
 'H': [],
 'HO2': [],
 'H2O2': [],
 'species_list': ['H2', 'O2', 'OH', 'O', 'H', 'HO2', 'H2O2'],
 'species_indices': array([0, 3, 4, 2, 1, 6, 7])}

# a) In the cell below, set the state of the



Now, we assemble a reactor network to manage our simulations.
To do this, we first make a Reactor from the Solution object, then placing the Reactor in a ReactorNetwork.

Several types of zero-dimensional reactors are available in Cantera.
Two main types you may be interested in are the constant-volume `ct.Reactor` and the constant-pressure
 `ct.ConstantPressureReactor`.
In this assignment, all simulations will be run at constant pressure using the latter reactor type.

In [44]:
# instantiate a reactor using the GRI 3.0 gas object defined above
reactor = ct.ConstPressureReactor(gas)

# instantiate a reactor network using our one and only reactor, which must be passed inside a list as an argument
r_net = ct.ReactorNet([reactor])

In order to run a kinetic simulation, we can `step` our reactor
